<a href="https://colab.research.google.com/github/tdiego74/TECO/blob/main/Copia_de_Categorizacion_de_Multas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from collections import defaultdict

# Descargar recursos necesarios de nltk
nltk.download('punkt')

# Ruta al archivo en Colab
file_path = 'https://raw.githubusercontent.com/tdiego74/TECO/main/motivos_multas.csv'

# Intentar cargar el archivo con codificación 'latin1' y manejar errores en las líneas
df = pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')

# Palabras clave y categorías predefinidas (usar minusculas en las palabras entre los corchetes)
palabras_clave_categorias = {
    "Semáforo": ["luces de semaforo", "luz de semaforo", "semáforo", "semaforo", "luz roja"],
    "Alcoholemia": ["alcoholemia", "alcohol"],
    "Exceso de Velocidad": ["exceso", "velocidad", "veloc."],
    "No Uso de Cinturón de Seguridad": ["cinturon", "cinturón", "cinturon de seguridad", "cinturón de seguridad", "correajes", "correajes de seguridad"],
    "No Respeto de Señales de Tránsito": ["señales", "giro", "girar", "carril", "bocacalle", "contramano", "sentido contrario"],
    "Estacionamiento Prohibido": ["estac.", "estacionar", "lugar prohibido", "reservado", "estacionamiento", "ticket", "doble fila", "triple fila", "ochava", "garage"],
    "Uso Indebido del Teléfono": ["telefonicos", "telefono", "celular", "mensaje de texto", "redactando", "enviando", "comunicacion", "comunicación"],
    "Evadir cobro de Peaje": ["peaje"],
    "Falta de Documentación": ["cedula", "vencida", "placas", "chapa", "patente", "vtv", "itv", "documentacion", "revisión técnica", "licencia", "documentación", "verificacion tecnica"],
    "Uso Incorrecto de Luces": ["luces", "uso de luces", "freno", "dirección", "retroceso", "niebla", "emergencia", "baja", "alta", "sucias", "encendidas", "apagadas"],
    "Obstrucción de Senda Peatonal": ["senda", "peatonal", "obstruir", "obstruccion", "circulación", "obstaculizar"],
    "Carga y Descarga Indebida": ["carga", "descarga", "fuera de hora", "hora/lugar prohibido"],
    "Otras Infracciones": ["maniobra peligrosa", "conducir peligroso", "no detenerse"]
}

# Definir la prioridad de las categorías
prioridad_categorias = [
    "Alcoholemia",
    "Semáforo",
    "Exceso de Velocidad",
    "No Uso de Cinturón de Seguridad",
    "Estacionamiento Prohibido",
    "No Respeto de Señales de Tránsito",
    "Uso Indebido del Teléfono",
    "Evadir cobro de Peaje",
    "Falta de Documentación",
    "Uso Incorrecto de Luces",
    "Obstrucción de Senda Peatonal",
    "Carga y Descarga Indebida",
    "Otras Infracciones"
]

# Función para asignar categoría y subcategoría con prioridad
def asignar_categoria(motivo):
    motivo_lower = motivo.lower()
    categorias_encontradas = set()

    for categoria, palabras_clave in palabras_clave_categorias.items():
        for palabra in palabras_clave:
            if palabra in motivo_lower:
                categorias_encontradas.add(categoria)
                break

    if categorias_encontradas:
        # Encontrar la categoría de mayor prioridad
        categoria_asignada = min(categorias_encontradas, key=lambda x: prioridad_categorias.index(x))
        subcategoria_asignada = next((palabra for palabra in palabras_clave_categorias[categoria_asignada] if palabra in motivo_lower), "")
    else:
        categoria_asignada = "Otras"
        subcategoria_asignada = ""

    return pd.Series([categoria_asignada, subcategoria_asignada])

# Aplicar la función a la columna de motivos
df[['Categoría', 'Subcategoría']] = df['Motivo'].apply(asignar_categoria)

# Identificar motivos sin categoría asignada y sugerir nuevas categorías
nuevos_motivos = df[df['Categoría'] == "Otras"]['Motivo'].unique()
nuevas_categorias_sugeridas = defaultdict(list)

for motivo in nuevos_motivos:
    motivo_lower = motivo.lower()
    for palabra in motivo_lower.split():
        nuevas_categorias_sugeridas[palabra].append(motivo)

# Guardar el resultado en un nuevo archivo CSV
df.to_csv("/content/motivos_multas_categorizados.csv", index=False, encoding='utf-8-sig')

# Guardar nuevas categorías sugeridas en un archivo CSV
nuevas_categorias_df = pd.DataFrame.from_dict(nuevas_categorias_sugeridas, orient='index').transpose()
nuevas_categorias_df.to_csv("/content/nuevas_categorias_sugeridas.csv", index=False, encoding='utf-8-sig')

print("Categorías asignadas y guardadas en 'motivos_multas_categorizados.csv'")
print("Nuevas categorías sugeridas guardadas en 'nuevas_categorias_sugeridas.csv'")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Categorías asignadas y guardadas en 'motivos_multas_categorizados.csv'
Nuevas categorías sugeridas guardadas en 'nuevas_categorias_sugeridas.csv'
